First of all, you should have a azure databricks workspace and a cluster to run code.

For migrating your local model on azure databricks, first compress your pickle files or model files into .jar files. For this you can also compress file into .zip then  convert it into .jar by writting on comand prompt

*mv model.zip model.jar*

Now upload this jar into your workspace by clicking import option on your working folder. After this open a notebook and decompress these file into tmp folder of your workspace. 

In [0]:
%sh 
rm -rf /tmp/model
mkdir /tmp/model

Now after creation unzip them in model folder

In [0]:
import zipfile
with zipfile.ZipFile("/dbfs/FileStore/jars/idOfYourModelJarFile-forest.jar","r") as zip_ref:
    print(zip_ref)
    zip_ref.extractall("/tmp/model")
with zipfile.ZipFile("/dbfs/FileStore/jars/idOfYourOtherRequiredJar-impColumns.jar","r") as zip_ref:
    print(zip_ref)
    zip_ref.extractall("/tmp/model")

load your model in your notebook

In [0]:
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.externals import joblib

#read model from the tmp path
model = joblib.load("/tmp/model/model.pkl")
impColumns = joblib.load("/tmp/model/impColumns.pkl")


Make prediction on the model

In [0]:
import sys
from sklearn.externals import joblib
import datetime
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import os

#Opening and reading file from url or take it from local
df_spark = spark.read.format("formatofFile").load(FILEPATH)
df = df_spark.toPandas()
#df.head()
    
#making predictions on data    
pred = model.predict_proba(df[impColumns]).max(axis=1)
pred_class = model.predict(df[impColumns])
df_output = pd.DataFrame()
df_output['UserId'] = df.UserId
df_output = df_output.assign(preb_prob=pred)
df_output = df_output.assign(classType=pred_class)
print(df_output)


    
    

Uploading outputted file at the location in dbfs

In [0]:
#Outputting file at required location in dbfs
outname = "test.txt"
outdir = '/dbfs/theLocationWhereYouWantToOutputYourFile/'
df_output.to_csv(outdir+outname, index=False, sep='\t',header=True)
print("File %s has been upload to" % outdir)
    